# orderbook의 데이터 형태 분석
{<br>
&nbsp;&nbsp;&nbsp;&nbsp;"type":"orderbook",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 토픽을 결정짓는 attr <br>
&nbsp;&nbsp;&nbsp;&nbsp;"code":"KRW-BTC",&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 종목(채결이었으면 카운팅 할 대상)<br>
&nbsp;&nbsp;&nbsp;&nbsp;"timestamp":1575553988269,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 업데이트 타임스템프 - 식별자의 가치가 있음<br>
&nbsp;&nbsp;&nbsp;&nbsp;"total_ask_size":12.60378903,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 현재 총 매도물량<br>
&nbsp;&nbsp;&nbsp;&nbsp;"total_bid_size":9.15384474,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 현재 총 매수물량<br>
&nbsp;&nbsp;&nbsp;&nbsp;"orderbook_units":&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 이 attr은 리스트를 가지는 게층형 구조 또는 nested 구조<br>
&nbsp;&nbsp;&nbsp;&nbsp;\[<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8703000.0,"bid_price":8695000.0,"ask_size":0.0311278,"bid_size":0.69556471},&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 매도1호가, 매수1호가 : 매도호가 중 가장 가격이 낮은 것과 매수호가 중 가장 가격이 높은 것  매도1호가 > 매수1호가  그리고 그 물량 <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8704000.0,"bid_price":8694000.0,"ask_size":0.002,"bid_size":0.17823952},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8706000.0,"bid_price":8693000.0,"ask_size":0.3395,"bid_size":0.10012},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8707000.0,"bid_price":8692000.0,"ask_size":0.19713255,"bid_size":0.42492},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8708000.0,"bid_price":8691000.0,"ask_size":0.04117598,"bid_size":0.86286356},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8709000.0,"bid_price":8690000.0,"ask_size":2.86058729,"bid_size":0.40441988},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8713000.0,"bid_price":8689000.0,"ask_size":2.0,"bid_size":0.67556021},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8715000.0,"bid_price":8688000.0,"ask_size":2.07204723,"bid_size":0.10012},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8716000.0,"bid_price":8687000.0,"ask_size":2.0288,"bid_size":0.10012},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8717000.0,"bid_price":8686000.0,"ask_size":0.869,"bid_size":1.57712},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8718000.0,"bid_price":8685000.0,"ask_size":0.5166,"bid_size":1.89979874},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8719000.0,"bid_price":8684000.0,"ask_size":1.0736,"bid_size":1.14239477},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8720000.0,"bid_price":8683000.0,"ask_size":0.09076608,"bid_size":0.40230335},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8722000.0,"bid_price":8682000.0,"ask_size":0.04931107,"bid_size":0.288},<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"ask_price":8724000.0,"bid_price":8680000.0,"ask_size":0.43214103,"bid_size":0.3023}<br>
&nbsp;&nbsp;&nbsp;&nbsp;],<br>
&nbsp;&nbsp;&nbsp;&nbsp;"stream_type":"SNAPSHOT"&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  어차피 스냅샷이니 필요없음<br>
}


In [8]:
!pip3 install confluent_kafka
!pip3 install websocket-client

    100% |################################| 204kB 1.4MB/s ta 0:00:01


In [9]:
from confluent_kafka import Producer
import sys


In [10]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

In [11]:
def pub_msg(producer,topic,msg):
    try:
        # Produce line (without newline)
        producer.produce(topic, msg, callback=delivery_report)
        producer.poll(0)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' %
                             len(producer))
    except Exception as e:
        print(e)

In [12]:
import websocket
import json

try:
    import thread
except ImportError:
    import _thread as thread
import time

global count 
global producer 
producer = Producer({'bootstrap.servers': 'localhost:9092,localhost:9093,localhost:9094'})

count = 0

def on_message(ws, message):
    global count 
    global producer 
    
    count += 1
    try:
        if count == 100000:
            ws.close()
        pub_msg(producer,'orderbook',message)
    except Exception:
        ws.close()

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    def run(*args):
        ws.send('[{"ticket":"koock2"},{"type":"orderbook","codes":["KRW-BTC","KRW-ETH","KRW-BCH","KRW-XRP","ETH-BCH","BTC-BCH"]}]')
    thread.start_new_thread(run, ())


if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://api.upbit.com/websocket/v1",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

--- request header ---
GET /websocket/v1 HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: api.upbit.com
Origin: http://api.upbit.com
Sec-WebSocket-Key: uQHvdjLS2S3o9P8/8Mn/Iw==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 05 Dec 2019 16:29:44 GMT
Content-Type: text/plain; charset=UTF-8
Connection: upgrade
Limit-By-IP: Yes
Remaining-Req: group=websocket; min=99; sec=4
upgrade: websocket
sec-websocket-accept: 2xluJ2/w2mqCgFzcufa7+66fJUc=
-----------------------
send: b'\x81\xf0\x7f\x16Y0$m{D\x16u2U\x0b4c\x12\x14y6S\x14${MSm{D\x06f<\x12E46B\x1bs+R\x10y2\x12S4:_\x1bs*\x12EM{{-Atr+U{\x1c]]\x0bgRS\rx]:{{-Atr<^{\x1c]]\x0bgRN\x0b`]:{u+^tr<^{\x1c]T\rsRT\x1ax]K$m'


Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

...

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to orderbook [0]
Message delivered to

send: b'\x88\x82\xf6=?\xe9\xf5\xd5'



### closed ###
